In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install albumentations

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import json
import os
from IPython.display import Image
import random
import glob
from albumentations import (
    Compose, OneOf, HorizontalFlip, VerticalFlip, ShiftScaleRotate,
    RandomBrightnessContrast, GaussNoise,HueSaturationValue
)

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
df = pd.read_csv('/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/annotations.csv')

In [6]:
def load_yolo_labels(label_path):
    """
    label_path: str, 라벨 1개 파일 경로
    YOLO 형식(.txt) 라벨을 불러와서,
    [ [cls, x_center, y_center, w, h], ... ] 형태의 리스트로 반환.
    모든 값은 float형, x_center 등은 0~1 정규화 상태여야 함.
    """
    boxes = []
    with open(label_path, "r") as f:
        for line in f.read().strip().splitlines():
            parts = line.split()
            cls_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            w = float(parts[3])
            h = float(parts[4])
            boxes.append([cls_id, x_center, y_center, w, h])
    return boxes


def save_yolo_labels(label_path, bbox_list):
    """
    label_path: str, 라벨 1개 파일 경로
    bbox_list: [ [cls, x_center, y_center, w, h], ... ]
    값을 텍스트로 저장.
    """
    with open(label_path, "w") as f:
        for cls_id, x_center, y_center, w, h in bbox_list:
            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")


## label이 존재하는 이미지만 리스트

내가 해야하는것.
데이터 증강을 위한 이미지와 라벨 1:1 매칭 시켜야함.
train label에 해당하는 이미지는 다있는것 같음.
이미지에 해당하는 label은 다없는것 같음.

실제로 train/labels가 데이터가 딱맞음.
labels에 해당하는 데이터 파일 이름만 가져오면됨.

In [73]:
ai02_img_path = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/ai02-level1-project/train/images'

train_img_path = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/train/images'
val_img_path = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/val/images'

train_labels_path = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/train/labels'
val_labels_path = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/val/labels'


label_file_list = os.listdir(train_labels_path)
label_file_name_list = []
for file in label_file_list:
  file = os.path.splitext(file)[0]
  label_file_name_list.append(file)

print(label_file_name_list)

['K-002483-003743-004378-023223_0_2_0_2_90_000_200', 'K-002483-003743-004378-023223_0_2_0_2_75_000_200', 'K-002483-003743-005886-006192_0_2_0_2_90_000_200', 'K-002483-003743-005886-006192_0_2_0_2_75_000_200', 'K-002483-003743-005886-012778_0_2_0_2_70_000_200', 'K-002483-003743-005886-012778_0_2_0_2_90_000_200', 'K-002483-003743-005886-022627_0_2_0_2_70_000_200', 'K-002483-003743-005886-022627_0_2_0_2_75_000_200', 'K-002483-003743-005886-023223_0_2_0_2_70_000_200', 'K-002483-003743-005886-022627_0_2_0_2_90_000_200', 'K-002483-003743-005886-023223_0_2_0_2_75_000_200', 'K-002483-003743-005886-023223_0_2_0_2_90_000_200', 'K-002483-003743-006192-012081_0_2_0_2_75_000_200', 'K-002483-003743-006192-012081_0_2_0_2_90_000_200', 'K-002483-003743-006192-013395_0_2_0_2_70_000_200', 'K-002483-003743-006192-013395_0_2_0_2_75_000_200', 'K-002483-003743-006192-013395_0_2_0_2_90_000_200', 'K-002483-003743-012081-012778_0_2_0_2_70_000_200', 'K-002483-003743-006192-022362_0_2_0_2_75_000_200', 'K-002483-0

## 150개 이하 클래스가 포함된 이미지 파일 객체
ex) {1: [1.png, 2.png], 2:[3.png, 4.png]}

In [74]:
"""
고유아이디 -> img_dl_idx
이미지 파일 명 -> img_imgfile
df 행을 순회하며, 고유 아이디에 해당하는 img_imgfile을 추가한다.
img_dl_idx: [img_imgfile, img_imgfile...]
"""
img_dl_idx_dict = {}
img_dl_idx_df = df[['img_dl_idx', 'img_imgfile']]
under_150_classes_obj = {}

# 클래스 당 이미지 파일 리스트 객체 생성
for _, row in img_dl_idx_df.iterrows():
    img_dl_idx = row['img_dl_idx']
    img_imgfile = row['img_imgfile']
    if img_dl_idx not in img_dl_idx_dict:
        img_dl_idx_dict[img_dl_idx] = []
    img_dl_idx_dict[img_dl_idx].append(img_imgfile)

# img_dl_idx_dict 클래스 73개 확인
print(f'img_dl_idx_dict 클래스 73개: {len(img_dl_idx_dict)}')

# 150개 이하 이미지 파일 클래스 객체
for key, value in img_dl_idx_dict.items():
    img_dl_idx_dict_len = len(value)

    if img_dl_idx_dict_len < 150:
        under_150_classes_obj[key] = value

print(f'150개 이하 클래스 객체 길이: {len(under_150_classes_obj)}')
print(f'150개 이하 클래스 객체: {under_150_classes_obj}')

img_dl_idx_dict 클래스 73개: 73
150개 이하 클래스 객체 길이: 69
150개 이하 클래스 객체: {16550: ['K-001900-010224-016551-031705_0_2_0_2_90_000_200.png', 'K-001900-010224-016551-031705_0_2_0_2_75_000_200.png', 'K-001900-010224-016551-031705_0_2_0_2_70_000_200.png', 'K-001900-010224-016551-033009_0_2_0_2_75_000_200.png', 'K-001900-010224-016551-033009_0_2_0_2_90_000_200.png', 'K-001900-016551-018110-021026_0_2_0_2_90_000_200.png', 'K-001900-016551-018110-021026_0_2_0_2_70_000_200.png', 'K-001900-016551-018110-027926_0_2_0_2_75_000_200.png', 'K-001900-016551-018110-027926_0_2_0_2_90_000_200.png', 'K-001900-016551-018110-027926_0_2_0_2_70_000_200.png', 'K-001900-016551-018110-029345_0_2_0_2_70_000_200.png', 'K-001900-016551-018110-029345_0_2_0_2_75_000_200.png', 'K-001900-016551-018110-029451_0_2_0_2_90_000_200.png', 'K-001900-016551-018110-029451_0_2_0_2_70_000_200.png', 'K-001900-016551-018110-031705_0_2_0_2_75_000_200.png', 'K-001900-016551-018110-031705_0_2_0_2_70_000_200.png', 'K-001900-016551-018110-03300

## YOLO 좌표 변환 코드

In [75]:
# ───────────────────────────────────────────────────────────────────
# 1) 확인해야 할 것: Albumentations는 바운딩박스를 (x_min, y_min, x_max, y_max) 형식으로 처리.
#    → 우리는 YOLO( x_center, y_center, w, h, 모두 0~1 )를 쓰므로 서로 변환해 줘야 함.
# ───────────────────────────────────────────────────────────────────

def yolo_to_alb_bbox(bbox_yolo, img_width, img_height):
    """
    YOLO 형식 → Albumentations 형식 (절대 좌표)
    bbox_yolo: [x_center, y_center, w, h] (0~1 정규화)
    반환: [x_min, y_min, x_max, y_max] (절대 픽셀 좌표)
    """
    x_center, y_center, w_norm, h_norm = bbox_yolo
    w = w_norm * img_width
    h = h_norm * img_height
    cx = x_center * img_width
    cy = y_center * img_height
    x_min = cx - w / 2
    y_min = cy - h / 2
    x_max = cx + w / 2
    y_max = cy + h / 2
    return [x_min, y_min, x_max, y_max]


def alb_to_yolo_bbox(bbox_alb, img_width, img_height):
    """
    Albumentations 형식 (절대 좌표) → YOLO 형식 (정규화)
    bbox_alb: [x_min, y_min, x_max, y_max] (절대 픽셀 좌표)
    반환: [x_center, y_center, w_norm, h_norm]
    """
    x_min, y_min, x_max, y_max = bbox_alb
    w = x_max - x_min
    h = y_max - y_min
    cx = x_min + w / 2
    cy = y_min + h / 2
    x_center = cx / img_width
    y_center = cy / img_height
    w_norm = w / img_width
    h_norm = h / img_height
    return [x_center, y_center, w_norm, h_norm]


# ───────────────────────────────────────────────────────────────────
# 2) 증강 파이프라인 정의 (YOLO용 bbox_params 설정)
# ───────────────────────────────────────────────────────────────────
augmentor = Compose(
    [
        OneOf([
            HorizontalFlip(p=1.0),  # 가로 뒤집기 (확률 100% 선택)
            VerticalFlip(p=1.0),    # 세로 뒤집기 (확률 100% 선택)
        ], p=0.5),                  # 둘 중 하나를 50% 확률로 적용

        # HueSaturationValue: hue_shift_limit=0 설정 → 색조 변화 없음
        # sat_shift_limit=40 범위 내에서 채도만 변경
        HueSaturationValue(
            hue_shift_limit=0,     # 색조 변화 없음
            sat_shift_limit=40,    # 채도 ±40
            val_shift_limit=40,     # 명도는 ±40
            p=0.7
        ),

        # 밝기·대비 변경 (±20% 범위)
        RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.2,
            p=0.7
        ),
    ],
    bbox_params={
        "format": "pascal_voc",       # [x_min, y_min, x_max, y_max]
        "label_fields": ["category_ids"],
        "min_visibility": 0.2         # 20% 미만으로 잘린 박스는 제거
    },
)

## 증강 코드

In [76]:
# ────────────────────────────────────────────────────────────────────────
# 1) 경로 설정
# ────────────────────────────────────────────────────────────────────────
IMAGE_DIR = "/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/ai02-level1-project/train/images" # 원본 .png 이미지들이 있는 폴더
LABEL_DIR = "/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/train/labels"                # 원본 YOLO .txt 라벨들이 있는 폴더
AUG_IMG_DIR   = "/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/augmented/images"        # 증강된 이미지를 저장할 폴더
AUG_LBL_DIR   = "/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/augmented/labels"        # 증강된 라벨을 저장할 폴더

os.makedirs(AUG_IMG_DIR, exist_ok=True)
os.makedirs(AUG_LBL_DIR, exist_ok=True)

# ───────────────────────────────────────────────────────────────────
# 2) 소수 클래스 대상 이미지 목록 수집, 150개 이하 클래스
# ───────────────────────────────────────────────────────────────────
all_basenames = [
    os.path.splitext(f)[0]
    for f in os.listdir(IMAGE_DIR)
    if f.endswith(".jpg") or f.endswith(".png")
]

# ───────────────────────────────────────────────────────────────────
# 3) 원하는 증강 복제 개수 설정
# ───────────────────────────────────────────────────────────────────
# 예: 전체 trainset의 0.3배만큼만 증강 파일을 만들고 싶다면:
num_to_generate = 150

# ────────────────────────────────────────────────────────────────────
# 6) 클래스별 증강 생성 루프
# ────────────────────────────────────────────────────────────────────
# 각 클래스별로 “현재 개수 = under_150_classes_obj[cls_id]의 리스트 길이” 계산
# 그리고 부족량만큼 증강
dup_counters = {cls_id: 0 for cls_id in under_150_classes_obj.keys()}

for cls_id, png_list in under_150_classes_obj.items():
    current_count = len(png_list)
    need = 200 - current_count
    if need <= 0:
        # 이미 200개 이상이면 건너뜀
        continue

    # “베이스명”만 추출 (예: 'K-001900-010224-016551-031705_0_2_0_2_90_000_200')
    base_list = [os.path.splitext(fname)[0] for fname in png_list]
    if len(base_list) == 0:
        raise ValueError(f"클래스 {cls_id}에 해당하는 원본 이미지가 없습니다.")

    print(f"클래스 {cls_id} → 현재 {current_count}개, 부족 {need}개 → 증강 시작")
    for _ in range(need):
        # 1) 무작위 원본 이미지 선택
        base = random.choice(base_list)
        img_path = os.path.join(IMAGE_DIR, base + ".png")
        lbl_path = os.path.join(LABEL_DIR, base + ".txt")

        # 2) 이미지 및 라벨 로드
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"이미지 로드 실패: {img_path}")
        h, w = img.shape[:2]
        raw_boxes = load_yolo_labels(lbl_path)  # [[cls, x_center, y_center, w, h],...]

        # 3) Albumentations용 bbox로 변환
        bboxes_alb = []
        category_ids = []
        for box in raw_boxes:
            cls0 = box[0]
            yolo_bbox = box[1:]  # [x_center, y_center, w, h]
            alb_bbox = yolo_to_alb_bbox(yolo_bbox, w, h)
            bboxes_alb.append(alb_bbox)
            category_ids.append(cls0)

        # 4) 증강 수행
        augmented = augmentor(
            image=img,
            bboxes=bboxes_alb,
            category_ids=category_ids
        )
        aug_img = augmented["image"]
        aug_bboxes_alb = augmented["bboxes"]
        aug_category_ids = augmented["category_ids"]

        # 5) Albumentations bbox → YOLO bbox로 복원
        new_bboxes = []
        for alb_bbox, cls0 in zip(aug_bboxes_alb, aug_category_ids):
            yolo_bbox = alb_to_yolo_bbox(alb_bbox, w, h)
            # 유효한 너비·높이인지 확인
            if yolo_bbox[2] <= 0 or yolo_bbox[3] <= 0:
                continue
            new_bboxes.append([cls0] + yolo_bbox)

        if not new_bboxes:
            # 박스가 전부 사라지면 재시도
            continue

        # 6) 증강 파일명 생성
        dup_counters[cls_id] += 1
        new_base = f"{base}_cls{cls_id}_dup{dup_counters[cls_id]}"

        # 7) 증강된 이미지 저장 (.png)
        out_img_path = os.path.join(AUG_IMG_DIR, new_base + ".png")
        cv2.imwrite(out_img_path, aug_img)

        # 8) 증강된 라벨 저장 (.txt)
        out_lbl_path = os.path.join(AUG_LBL_DIR, new_base + ".txt")
        save_yolo_labels(out_lbl_path, new_bboxes)

    print(f"클래스 {cls_id} 증강 완료 → 최종 200개 확보")

print("─── 모든 클래스 증강 데이터 생성 완료 ───")

클래스 16550 → 현재 139개, 부족 61개 → 증강 시작


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/train/labels/K-001900-010224-016551-031705_0_2_0_2_70_000_200.txt'